In [1]:
import pynq
import numpy as np
from tqdm import tqdm
import time
#from sklearn.metrics import average_precision_score, roc_auc_score

# 通过PYNQ将合成电路写入FPGA中
ol_loda = pynq.Overlay('./SCD_Inter_Quarter_16161616.bit')
# ol_loda = pynq.Overlay('./SCD_Inter_Half_16161616.bit')
mSCD = ol_loda.model_SCD_0
mM2M = ol_loda.Multi2Multi_0
mM2O = ol_loda.Multi2One_0

CONTROL_REGISTER = 0x0
mM2O.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0)
}

In [2]:
def _load_via_txt(path):
    X = np.loadtxt(path, delimiter=",")
    return X

def get_data(data_file):
    #data_path = os.path.join(data_base_path, data_file)
    data_path = data_file
    if ".mat" in data_file:
        from scipy.io import loadmat
        f = loadmat(data_path)
        X = f['X']
    else:
        X = _load_via_txt(data_path)
    return X

X_train = get_data("./deepsig.dat")  # Load Aryhytmia data.
print("X_train.shape = ",X_train.shape)

X_train.shape =  (2048, 2)


In [3]:
# 使用AXI DMA进行存储器传输
dma_i = ol_loda.axi_dma_0.sendchannel
dma_q = ol_loda.axi_dma_1.sendchannel
dma_v = ol_loda.axi_dma_2.recvchannel
dma_a = ol_loda.axi_dma_3.recvchannel
dma_f = ol_loda.axi_dma_4.recvchannel
# 处理大量数据
N = 2048 # X_train.shape[0]
Nout = 807#32768
NSamples = 1000

buff_x = pynq.allocate(shape=(NSamples*N,), dtype=np.float32)
buff_y = pynq.allocate(shape=(NSamples*N,), dtype=np.float32)
buff_value = pynq.allocate(shape=(NSamples*Nout,), dtype=np.float32)
buff_al = pynq.allocate(shape=(NSamples*Nout,), dtype=np.int16)
buff_fl = pynq.allocate(shape=(NSamples*Nout,), dtype=np.int16)

# print("X_train[0:N] = ",X_train[0:N][:])


for j in range (NSamples):
    buff_x[j*N:(j+1)*N] = X_train[0:N,0]
    buff_y[j*N:(j+1)*N] = X_train[0:N,1]
    

In [4]:
start = time.time()
mSCD.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
mM2M.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
mM2O.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0

dma_i.transfer(buff_x)
dma_q.transfer(buff_y)
dma_v.transfer(buff_value)
dma_a.transfer(buff_al)
dma_f.transfer(buff_fl)
dma_i.wait()
dma_q.wait()
# dma_r.wait()
end = time.time()
mSCD.write(CONTROL_REGISTER,0x00) # stop
mM2M.write(CONTROL_REGISTER,0x00) # stop
mM2O.write(CONTROL_REGISTER,0x00) # stop
time_cost= end-start
print('=> Running on FPGA')
print(time_cost)
print()

=> Running on FPGA
0.08783221244812012



In [5]:
# print("buff_y[0:N] = ",buff_y[0:10])
# print("buff_y[-1] = ",buff_y[-1])
# del buff_x, buff_y

In [6]:
buff_value[0]

0.106536865

In [7]:
x = np.arange(0,809)
for i in x:
    print(i,":",buff_value[i],",",buff_al[i],",",buff_fl[i])
#     print(i,";",buff_z[i],",")

0 : 0.106536865 , 1 , 8
1 : 0.050323486 , 8 , 10
2 : 0.05001831 , 24 , 8
3 : 0.061035156 , 206 , 8
4 : 0.099243164 , 258 , 8
5 : 0.0552063 , 463 , 8
6 : 0.052734375 , 485 , 8
7 : 0.06472778 , 1017 , 8
8 : 0.050689697 , 1020 , 8
9 : 0.07803345 , 1513 , 8
10 : 0.07223511 , 1734 , 8
11 : 0.055603027 , 1752 , 8
12 : 0.23672485 , 1758 , 8
13 : 0.05722046 , 1764 , 8
14 : 0.06851196 , 1804 , 8
15 : 0.051696777 , 1841 , 8
16 : 0.053588867 , 1846 , 8
17 : 0.06008911 , 1849 , 8
18 : 0.056243896 , 1860 , 8
19 : 0.053344727 , 1868 , 8
20 : 0.060791016 , 1874 , 10
21 : 0.059326172 , 1882 , 8
22 : 0.095825195 , 1895 , 8
23 : 0.051757812 , 1930 , 10
24 : 0.08465576 , 1970 , 8
25 : 0.053894043 , 1978 , 8
26 : 0.09802246 , 1979 , 8
27 : 0.0541687 , 1990 , 8
28 : 0.103271484 , 1994 , 8
29 : 0.07015991 , 2073 , 6
30 : 0.05117798 , 2109 , 4
31 : 0.06048584 , 2143 , 6
32 : 0.051818848 , 2187 , 6
33 : 0.06832886 , 2197 , 6
34 : 0.054382324 , 2481 , 8
35 : 0.06417847 , 3186 , 8
36 : 0.06100464 , 3317 , 8
37 

348 : 0.051116943 , 10816 , 8
349 : 0.08114624 , 10866 , 8
350 : 0.0697937 , 11017 , 8
351 : 0.07312012 , 11166 , 8
352 : 0.05102539 , 11266 , 10
353 : 0.061645508 , 11742 , 8
354 : 0.10498047 , 12018 , 8
355 : 0.30581665 , 12153 , 8
356 : 0.08413696 , 12159 , 8
357 : 0.064971924 , 12162 , 8
358 : 0.09335327 , 12172 , 8
359 : 0.06390381 , 12184 , 8
360 : 0.08035278 , 12183 , 8
361 : 0.057769775 , 12188 , 8
362 : 0.05871582 , 12194 , 8
363 : 0.057800293 , 12198 , 8
364 : 0.06329346 , 12219 , 8
365 : 0.064819336 , 12221 , 10
366 : 0.06994629 , 12246 , 8
367 : 0.050933838 , 12245 , 8
368 : 0.08416748 , 12271 , 8
369 : 0.072631836 , 12279 , 8
370 : 0.112701416 , 12286 , 8
371 : 0.0736084 , 12297 , 6
372 : 0.069732666 , 12304 , 8
373 : 0.06265259 , 12304 , 10
374 : 0.052246094 , 12316 , 8
375 : 0.07211304 , 12351 , 6
376 : 0.08496094 , 12376 , 6
377 : 0.06970215 , 12377 , 8
378 : 0.05984497 , 12417 , 8
379 : 0.050750732 , 12482 , 8
380 : 0.10369873 , 12921 , 8
381 : 0.0579834 , 12924 , 8
38

633 : 0.052764893 , 13423 , 6
634 : 0.086364746 , 13424 , 8
635 : 0.052825928 , 13451 , 8
636 : 0.11444092 , 13726 , 8
637 : 0.06738281 , 13774 , 8
638 : 0.07644653 , 13780 , 8
639 : 0.055419922 , 13938 , 8
640 : 0.05331421 , 14142 , 8
641 : 0.052124023 , 14144 , 8
642 : 0.11077881 , 14241 , 8
643 : 0.24822998 , 14338 , 8
644 : 0.06390381 , 14350 , 8
645 : 0.06335449 , 14352 , 10
646 : 0.0602417 , 14352 , 8
647 : 0.057495117 , 14353 , 8
648 : 0.06097412 , 14364 , 8
649 : 0.0960083 , 14386 , 8
650 : 0.103393555 , 14411 , 8
651 : 0.050231934 , 14426 , 8
652 : 0.05609131 , 14430 , 8
653 : 0.060455322 , 14467 , 6
654 : 0.062194824 , 14479 , 6
655 : 0.051635742 , 14494 , 6
656 : 0.06793213 , 14504 , 6
657 : 0.06213379 , 14505 , 6
658 : 0.0597229 , 14526 , 8
659 : 0.056671143 , 14538 , 8
660 : 0.07733154 , 14617 , 8
661 : 0.116119385 , 15042 , 8
662 : 0.052337646 , 15107 , 8
663 : 0.06588745 , 15121 , 8
664 : 0.055664062 , 15274 , 8
665 : 0.096221924 , 15273 , 8
666 : 0.05960083 , 15325 , 8


In [8]:
import re
import math
dat_file = r"sparse_SCD_float_quarter16161616.dat"   
dm = []
al = []
fl = []
with open(dat_file, 'r') as file:
    text = file.read()
    data = re.split(',|\n', text)
    for i in range(int((len(data)-1)/3)):
        dm.append(float(data[i*3]))
        al.append(int(data[i*3+1]))
        fl.append(int(data[i*3+2]))

In [9]:
rdm = np.zeros(len(dm))
ral = np.zeros(len(dm))
rfl = np.zeros(len(dm))

In [10]:
for i in range(len(dm)):
    for j in range(len(np.where(al==buff_al[i])[-1])):
        if fl[np.where(al==buff_al[i])[0][j]] == buff_fl[i]:
#             print(i)
#             print(buff_al[i])
#             print(np.where(al==buff_al[i])[0][j])
#             print(fl[np.where(al==buff_al[i])[0][j]])
#             print(buff_fl[i])
#             print(buff_value[i])
#             print(dm[np.where(al==buff_al[i])[0][j]])
#             print('------------')
            rdm[i]=dm[np.where(al==buff_al[i])[0][j]]
            ral[i]=al[np.where(al==buff_al[i])[0][j]]
            rfl[i]=fl[np.where(al==buff_al[i])[0][j]]

In [11]:
Psignal = np.sum(np.power(rdm,2))
# Pnoise  = np.var(rdm-buff_value[0:len(rdm)])
Pnoise  = np.sum(np.power(rdm-buff_value[0:len(rdm)],2))
print("The SQNR is: " , 10*math.log10(Psignal/Pnoise))

The SQNR is:  73.65948713252413


In [12]:
Psignal = np.sum(np.power(rdm,2))
Pnoise  = np.var(rdm-buff_value[0:len(rdm)])
print("The SQNR is: " , 10*math.log10(Psignal/Pnoise))

The SQNR is:  97.42399665394498


In [13]:
for i in range(len(dm)):
    print(str(i)+':'+str(rdm[i])+','+str(buff_value[i]))

0:0.106570199,0.106536865
1:0.050304554,0.050323486
2:0.050025627,0.05001831
3:0.061018914,0.061035156
4:0.099229634,0.099243164
5:0.055203404,0.0552063
6:0.052717499,0.052734375
7:0.064704522,0.06472778
8:0.050693706,0.050689697
9:0.078040242,0.07803345
10:0.072224282,0.07223511
11:0.055587776,0.055603027
12:0.2367336,0.23672485
13:0.057224158,0.05722046
14:0.068532616,0.06851196
15:0.051677741,0.051696777
16:0.053572781,0.053588867
17:0.060099266,0.06008911
18:0.056231719,0.056243896
19:0.053362302,0.053344727
20:0.060780633,0.060791016
21:0.059308823,0.059326172
22:0.095874965,0.095825195
23:0.051770981,0.051757812
24:0.084678628,0.08465576
25:0.053900957,0.053894043
26:0.098013066,0.09802246
27:0.0541641,0.0541687
28:0.103276707,0.103271484
29:0.070137717,0.07015991
30:0.051177617,0.05117798
31:0.060501561,0.06048584
32:0.051826145,0.051818848
33:0.068356007,0.06832886
34:0.054369535,0.054382324
35:0.064157516,0.06417847
36:0.060996469,0.06100464
37:0.067022458,0.0670166
38:0.06895

In [14]:
np.max(rdm-buff_value[0:len(rdm)])

PynqBuffer(4.97696875e-05)

In [15]:
math.log2(np.max(rdm-buff_value[0:len(rdm)]))

-14.294373145963299

In [16]:
np.power([1,2,3],2)

array([1, 4, 9])

In [17]:
ral-buff_al[0:len(ral)]

PynqBuffer([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [18]:
rfl-buff_fl[0:len(rfl)]

PynqBuffer([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 